In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf

In [2]:
import os
os.listdir('C:\\newBirddataset')

['images']

In [3]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + val_split + test_split) == 1, "Splits must sum to 1."

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * len(ds))
    val_size = int(val_split * len(ds))

    train_ds = ds.take(train_size)
    remaining = ds.skip(train_size)
    val_ds = remaining.take(val_size)
    test_ds = remaining.skip(val_size)

    return train_ds, val_ds, test_ds


In [4]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    'C:\\newBirddataset\\images',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224),
)
labels=dataset.class_names

Found 11788 files belonging to 200 classes.


In [5]:
# Get class names from the dataset
class_names = dataset.class_names
#splitting in train,test and validation
train,test,validation=get_dataset_partitions_tf(dataset)
# Create a lookup layer to convert class names to integers
label_lookup = tf.keras.layers.StringLookup(vocabulary=class_names, num_oov_indices=0)




In [6]:
train

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [7]:

# Define the data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal_and_vertical'),  # Randomly flip images horizontally and vertically
    tf.keras.layers.RandomRotation(0.2),  # Randomly rotate images by 20%
    tf.keras.layers.RandomZoom(0.2),  # Randomly zoom into images
    tf.keras.layers.RandomContrast(0.1),  # Randomly adjust contrast
])

# Apply augmentation to the dataset
augmented_train = train.map(lambda x, y: (data_augmentation(x, training=True), y))

# Now you can use augmented_dataset for training


In [8]:
# Convert labels to integer indices
augmented_train = augmented_train.map(lambda x, y: (x, y))
validation = validation.map(lambda x, y: (x, y))
test = test.map(lambda x, y: (x, y))


In [9]:
labels

['001.Black_footed_Albatross',
 '002.Laysan_Albatross',
 '003.Sooty_Albatross',
 '004.Groove_billed_Ani',
 '005.Crested_Auklet',
 '006.Least_Auklet',
 '007.Parakeet_Auklet',
 '008.Rhinoceros_Auklet',
 '009.Brewer_Blackbird',
 '010.Red_winged_Blackbird',
 '011.Rusty_Blackbird',
 '012.Yellow_headed_Blackbird',
 '013.Bobolink',
 '014.Indigo_Bunting',
 '015.Lazuli_Bunting',
 '016.Painted_Bunting',
 '017.Cardinal',
 '018.Spotted_Catbird',
 '019.Gray_Catbird',
 '020.Yellow_breasted_Chat',
 '021.Eastern_Towhee',
 '022.Chuck_will_Widow',
 '023.Brandt_Cormorant',
 '024.Red_faced_Cormorant',
 '025.Pelagic_Cormorant',
 '026.Bronzed_Cowbird',
 '027.Shiny_Cowbird',
 '028.Brown_Creeper',
 '029.American_Crow',
 '030.Fish_Crow',
 '031.Black_billed_Cuckoo',
 '032.Mangrove_Cuckoo',
 '033.Yellow_billed_Cuckoo',
 '034.Gray_crowned_Rosy_Finch',
 '035.Purple_Finch',
 '036.Northern_Flicker',
 '037.Acadian_Flycatcher',
 '038.Great_Crested_Flycatcher',
 '039.Least_Flycatcher',
 '040.Olive_sided_Flycatcher',
 '

In [10]:
for i, name in enumerate(class_names):
    label_int = label_lookup(tf.constant(name)).numpy()
    print(f"Class name: {name}, Integer label: {label_int}")

Class name: 001.Black_footed_Albatross, Integer label: 0
Class name: 002.Laysan_Albatross, Integer label: 1
Class name: 003.Sooty_Albatross, Integer label: 2
Class name: 004.Groove_billed_Ani, Integer label: 3
Class name: 005.Crested_Auklet, Integer label: 4
Class name: 006.Least_Auklet, Integer label: 5
Class name: 007.Parakeet_Auklet, Integer label: 6
Class name: 008.Rhinoceros_Auklet, Integer label: 7
Class name: 009.Brewer_Blackbird, Integer label: 8
Class name: 010.Red_winged_Blackbird, Integer label: 9
Class name: 011.Rusty_Blackbird, Integer label: 10
Class name: 012.Yellow_headed_Blackbird, Integer label: 11
Class name: 013.Bobolink, Integer label: 12
Class name: 014.Indigo_Bunting, Integer label: 13
Class name: 015.Lazuli_Bunting, Integer label: 14
Class name: 016.Painted_Bunting, Integer label: 15
Class name: 017.Cardinal, Integer label: 16
Class name: 018.Spotted_Catbird, Integer label: 17
Class name: 019.Gray_Catbird, Integer label: 18
Class name: 020.Yellow_breasted_Chat, 

In [11]:
len(augmented_train )

295

In [12]:
len(train), len(test), len(validation)

(295, 36, 38)

In [13]:
# Normalize pixel values to the range [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Apply normalization to the dataset
normalized_train = augmented_train.map(lambda x, y: (normalization_layer(x), y))

In [14]:
normalized_test = test.map(lambda x, y: (normalization_layer(x), y))

In [15]:
normalized_validation = validation.map(lambda x, y: (normalization_layer(x), y))

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(shape=(224, 224, 3)),
    tf.keras.layers.BatchNormalization(),
    
    # First Convolutional Block
    tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(1, 1), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Second Convolutional Block
    tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(2, 2), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Third Convolutional Block
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1)),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),  # Pooling (2,2)
    
    # Fourth Convolutional Block
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),  # Pooling (2,2)
    
    tf.keras.layers.Dropout(0.5),
    
    # Fifth Convolutional Block
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same'),  # Corrected typo here
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),  # Pooling (2,2)
    
    tf.keras.layers.Dropout(0.5),
    
    # Fully Connected Layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(200, activation='softmax')  # Output layer with 525 classes
])

# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])


C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ batch_normalization                  │ (None, 224, 224, 3)         │              12 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 224, 224, 32)        │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 224, 224, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 224, 224, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 56, 56, 32)          │          25,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 56, 56, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 56, 56, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 28, 28, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 26, 26, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 26, 26, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 26, 26, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 13, 13, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 13, 13, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 6, 6, 128)           │          73,8

 Total params: 332,020 (1.27 MB)

 Trainable params: 331,374 (1.26 MB)

 Non-trainable params: 646 (2.52 KB)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)

# Train with callbacks
history = model.fit(
    normalized_train,
    epochs=10,
    validation_data=normalized_validation,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 701s 2s/step - accuracy: 0.0057 - loss: 5.4995 - val_accuracy: 0.0000e+00 - val_loss: 5.8114
Epoch 2/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 539s 2s/step - accuracy: 0.0100 - loss: 5.2378 - val_accuracy: 0.0000e+00 - val_loss: 5.2766
Epoch 3/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 749s 2s/step - accuracy: 0.0118 - loss: 5.1324 - val_accuracy: 0.0000e+00 - val_loss: 5.1018
Epoch 4/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 521s 2s/step - accuracy: 0.0199 - loss: 4.9992 - val_accuracy: 0.0625 - val_loss: 5.0855
Epoch 5/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 5433s 18s/step - accuracy: 0.0238 - loss: 4.8014 - val_accuracy: 0.0312 - val_loss: 4.4897
Epoch 6/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 1219s 4s/step - accuracy: 0.0331 - loss: 4.6032 - val_accuracy: 0.0000e+00 - val_loss: 4.4631
Epoch 7/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 1994s 6s/step - accuracy: 0.0531 - loss: 4.3919 - val_accuracy: 0.0938 - val_loss: 3.9790
Epoch 8/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 913s 3s/step - accuracy: 0.0548 - los


KeyboardInterrupt



In [12]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(shape=(224, 224, 3)),
    tf.keras.layers.BatchNormalization(),
    
    # First Convolutional Block
    tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), strides=(2, 2), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
    
    # Second Convolutional Block
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same'),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
  
    
    # Third Convolutional Block
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1),kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),

    # fourth Convolutional Block
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1),kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Fifth Convolutional Block
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),

    # Sixth Convolutional Block
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same',kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Corrected typo here
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),

    #seventh Convolutional Block
     tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same',kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Corrected typo here
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
   
    #eight Convolutional Block
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same',kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Corrected typo here
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    
    # Ninth Convolutional Block
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same',kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Corrected typo here
    tf.keras.layers.LeakyReLU(alpha=0.01),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),  # Pooling (2,2)
    
    tf.keras.layers.Dropout(0.5),
    
    # Fully Connected Layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(200, activation='softmax')  # Output layer with 200 classes
])

C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [13]:
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ batch_normalization                  │ (None, 224, 224, 3)         │              12 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 112, 112, 32)        │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 112, 112, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 112, 112, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 110, 110, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 110, 110, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 110, 110, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 108, 108, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 108, 108, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 108, 108, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 54, 54, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 54, 54, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 54, 54, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 54, 54, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 3,030,196 (11.56 MB)

 Trainable params: 3,029,166 (11.56 MB)

 Non-trainable params: 1,030 (4.02 KB)

In [16]:
model2.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])


In [17]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
# Train with callbacks
history = model2.fit(
    normalized_train,
    epochs=30,
    validation_data=normalized_validation,
    callbacks=[early_stopping, model_checkpoint,lr_schedule]
)


Epoch 1/30
295/295 ━━━━━━━━━━━━━━━━━━━━ 1611s 5s/step - accuracy: 0.0048 - loss: 6.0466 - val_accuracy: 0.0041 - val_loss: 14.2746 - learning_rate: 0.0010
Epoch 2/30
295/295 ━━━━━━━━━━━━━━━━━━━━ 1644s 5s/step - accuracy: 0.0051 - loss: 5.6258 - val_accuracy: 0.0017 - val_loss: 12.6070 - learning_rate: 0.0010
Epoch 3/30
295/295 ━━━━━━━━━━━━━━━━━━━━ 1689s 6s/step - accuracy: 0.0043 - loss: 5.5499 - val_accuracy: 0.0066 - val_loss: 18.1829 - learning_rate: 0.0010
Epoch 4/30
295/295 ━━━━━━━━━━━━━━━━━━━━ 1941s 6s/step - accuracy: 0.0051 - loss: 5.5784 - val_accuracy: 0.0074 - val_loss: 5.5973 - learning_rate: 0.0010
Epoch 5/30
 64/295 ━━━━━━━━━━━━━━━━━━━━ 2:11:37 34s/step - accuracy: 0.0055 - loss: 5.5693

KeyboardInterrupt: 

Using Transfer Learning which utilizes pre-trained models that have been trained on bigger datasets which helps them to learn features and this features can be utilized for the task of bird species classification

In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load pre-trained MobileNetV2 without the top classification layer
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),  # Your image size
    include_top=False,  # Remove the final dense layers (ImageNet's output)
    weights='imagenet'  # Use pre-trained ImageNet weights
)

# Freeze the base model layers (so they are not updated during training)
base_model.trainable = False

# Create your model and add custom layers on top of the base model
trmodel = models.Sequential([
    base_model,  # Pre-trained MobileNetV2 layers
    layers.GlobalAveragePooling2D(),  # Reduce feature map to a vector by averaging
    layers.Dense(256, activation='relu'),  # Add a dense layer for learning
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(200, activation='softmax')  # Final layer for your 200 bird species
])

# Compile the model
trmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model summary
trmodel.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 200)                 │          51,400 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,637,320 (10.06 MB)

 Trainable params: 379,336 (1.45 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [14]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_transfer_lr_model.keras', save_best_only=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
# Train with callbacks
history = trmodel.fit(
    normalized_train,
    epochs=25,
    validation_data=normalized_validation,
    callbacks=[early_stopping, model_checkpoint,lr_schedule]
)

Epoch 1/25
295/295 ━━━━━━━━━━━━━━━━━━━━ 465s 1s/step - accuracy: 0.0126 - loss: 5.2774 - val_accuracy: 0.1069 - val_loss: 4.2033 - learning_rate: 0.0010
Epoch 2/25
295/295 ━━━━━━━━━━━━━━━━━━━━ 462s 1s/step - accuracy: 0.0599 - loss: 4.3711 - val_accuracy: 0.2333 - val_loss: 3.4338 - learning_rate: 0.0010
Epoch 3/25
295/295 ━━━━━━━━━━━━━━━━━━━━ 474s 1s/step - accuracy: 0.1138 - loss: 3.8716 - val_accuracy: 0.2870 - val_loss: 3.0218 - learning_rate: 0.0010
Epoch 4/25
295/295 ━━━━━━━━━━━━━━━━━━━━ 445s 1s/step - accuracy: 0.1321 - loss: 3.6410 - val_accuracy: 0.3520 - val_loss: 2.7073 - learning_rate: 0.0010
Epoch 5/25
295/295 ━━━━━━━━━━━━━━━━━━━━ 409s 1s/step - accuracy: 0.1589 - loss: 3.4847 - val_accuracy: 0.4005 - val_loss: 2.5358 - learning_rate: 0.0010
Epoch 6/25
295/295 ━━━━━━━━━━━━━━━━━━━━ 426s 1s/step - accuracy: 0.1796 - loss: 3.3491 - val_accuracy: 0.3602 - val_loss: 2.4499 - learning_rate: 0.0010
Epoch 7/25
295/295 ━━━━━━━━━━━━━━━━━━━━ 415s 1s/step - accuracy: 0.2023 - loss: 3.

In [12]:
from tensorflow.keras.models import load_model

# Load the model from the latest checkpoint if saved
trmodelcontinue = load_model('best_transfer_lr_model.keras')


In [13]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_transfer_lr_model.keras', save_best_only=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)
history = trmodelcontinue.fit(
    normalized_train,
    epochs=35,  # Total epochs (25 already done + 10 more)
    initial_epoch=25,  # Start from epoch 26
    validation_data=normalized_validation,
    callbacks=[early_stopping, model_checkpoint, lr_schedule]
)

Epoch 26/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 418s 1s/step - accuracy: 0.3065 - loss: 2.6671 - val_accuracy: 0.5321 - val_loss: 1.7851 - learning_rate: 5.0000e-04
Epoch 27/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 438s 1s/step - accuracy: 0.3135 - loss: 2.6378 - val_accuracy: 0.5008 - val_loss: 1.8176 - learning_rate: 5.0000e-04
Epoch 28/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 405s 1s/step - accuracy: 0.3068 - loss: 2.6320 - val_accuracy: 0.5132 - val_loss: 1.8575 - learning_rate: 5.0000e-04
Epoch 29/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 394s 1s/step - accuracy: 0.3174 - loss: 2.6405 - val_accuracy: 0.5214 - val_loss: 1.7695 - learning_rate: 5.0000e-04
Epoch 30/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 398s 1s/step - accuracy: 0.3034 - loss: 2.6912 - val_accuracy: 0.5296 - val_loss: 1.7361 - learning_rate: 5.0000e-04
Epoch 31/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 396s 1s/step - accuracy: 0.3106 - loss: 2.6220 - val_accuracy: 0.4984 - val_loss: 1.8510 - learning_rate: 5.0000e-04
Epoch 32/35
295/295 ━━━━━━━━━━━━━━━━━━━━ 426s 1s/ste

In [18]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('best_transfer_lr_model.keras')

# Unfreeze only the last few layers of MobileNet for fine-tuning
# Assuming 'mobilenet' is part of the model
mobilenet = trmodel.get_layer('mobilenetv2_1.00_224')  # Change this if needed

# Unfreeze the last few layers
for layer in mobilenet.layers[-10:]:  # Adjust number of layers to unfreeze
    layer.trainable = True

# Compile the model again after unfreezing layers
trmodel.compile(
    optimizer=Adam(learning_rate=1e-4),  # You may want to lower the learning rate for fine-tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=45,  # Continue training for 10 more epochs
    initial_epoch=35,
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, lr_schedule]
)


Epoch 36/45
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2448 - loss: 3.0638
Epoch 36: val_loss improved from inf to 1.85154, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 696s 2s/step - accuracy: 0.2448 - loss: 3.0635 - val_accuracy: 0.4707 - val_loss: 1.8515 - learning_rate: 1.0000e-04
Epoch 37/45
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2846 - loss: 2.8270
Epoch 37: val_loss improved from 1.85154 to 1.73830, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 729s 2s/step - accuracy: 0.2846 - loss: 2.8271 - val_accuracy: 0.5230 - val_loss: 1.7383 - learning_rate: 1.0000e-04
Epoch 38/45
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2917 - loss: 2.7622
Epoch 38: val_loss did not improve from 1.73830
295/295 ━━━━━━━━━━━━━━━━━━━━ 704s 2s/step - accuracy: 0.2917 - loss: 2.7622 - val_accuracy: 0.4844 - val_loss: 1.8004 - learning_rate: 1.0000e-04
Epoch 39/45
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step 

In [24]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the best saved model
model = load_model('best_transfer_finetune_model.keras')


model.compile(
    optimizer=Adam(learning_rate=5e-5),  # Continue with the reduced learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model for more epochs
history = model.fit(
    normalized_train,
    # Your training dataset
    epochs=10,
    # Add 10 more epochs, adjust as needed
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, lr_schedule]
)


Epoch 1/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3379 - loss: 2.5077
Epoch 1: val_loss improved from inf to 1.52895, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 795s 2s/step - accuracy: 0.3379 - loss: 2.5076 - val_accuracy: 0.5543 - val_loss: 1.5289 - learning_rate: 5.0000e-05
Epoch 2/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3600 - loss: 2.4233
Epoch 2: val_loss improved from 1.52895 to 1.50805, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 624s 2s/step - accuracy: 0.3599 - loss: 2.4234 - val_accuracy: 0.5781 - val_loss: 1.5081 - learning_rate: 5.0000e-05
Epoch 3/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3488 - loss: 2.4391
Epoch 3: val_loss did not improve from 1.50805
295/295 ━━━━━━━━━━━━━━━━━━━━ 624s 2s/step - accuracy: 0.3488 - loss: 2.4391 - val_accuracy: 0.5683 - val_loss: 1.5451 - learning_rate: 5.0000e-05
Epoch 4/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accur

In [1]:
from tensorflow.keras.models import load_model

# Load the model from the latest checkpoint if saved
trmodelcontinue = load_model('best_transfer_finetune_model.keras')

In [15]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

history = trmodelcontinue.fit(
    normalized_train,
    epochs=10,  # Total 10 more)
    validation_data=normalized_validation,
    callbacks=[early_stopping, model_checkpoint, lr_schedule]
)

Epoch 1/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3594 - loss: 2.4012
Epoch 1: val_loss improved from inf to 1.49719, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 675s 2s/step - accuracy: 0.3594 - loss: 2.4012 - val_accuracy: 0.5781 - val_loss: 1.4972 - learning_rate: 5.0000e-05
Epoch 2/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3620 - loss: 2.3944
Epoch 2: val_loss improved from 1.49719 to 1.46497, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 699s 2s/step - accuracy: 0.3620 - loss: 2.3944 - val_accuracy: 0.5778 - val_loss: 1.4650 - learning_rate: 5.0000e-05
Epoch 3/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3656 - loss: 2.3775
Epoch 3: val_loss did not improve from 1.46497
295/295 ━━━━━━━━━━━━━━━━━━━━ 714s 2s/step - accuracy: 0.3656 - loss: 2.3775 - val_accuracy: 0.5617 - val_loss: 1.5406 - learning_rate: 5.0000e-05
Epoch 4/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accur

In [12]:
from tensorflow.keras.models import load_model

# Load the model from the latest checkpoint if saved
trmodelcontinue = load_model('best_transfer_finetune_model.keras')

In [13]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

history = trmodelcontinue.fit(
    normalized_train,
    epochs=10,  # Total 10 more)
    validation_data=normalized_validation,
    callbacks=[early_stopping, model_checkpoint, lr_schedule]
)

Epoch 1/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3757 - loss: 2.3526
Epoch 1: val_loss improved from inf to 1.42424, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 480s 1s/step - accuracy: 0.3757 - loss: 2.3525 - val_accuracy: 0.6151 - val_loss: 1.4242 - learning_rate: 5.0000e-05
Epoch 2/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3748 - loss: 2.3509
Epoch 2: val_loss did not improve from 1.42424
295/295 ━━━━━━━━━━━━━━━━━━━━ 468s 1s/step - accuracy: 0.3748 - loss: 2.3508 - val_accuracy: 0.6179 - val_loss: 1.4433 - learning_rate: 5.0000e-05
Epoch 3/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 993ms/step - accuracy: 0.3781 - loss: 2.3038
Epoch 3: val_loss improved from 1.42424 to 1.34075, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 430s 1s/step - accuracy: 0.3780 - loss: 2.3038 - val_accuracy: 0.6472 - val_loss: 1.3407 - learning_rate: 5.0000e-05
Epoch 4/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - ac

In [14]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('best_transfer_finetune_model.keras')

# Unfreeze only the last few layers of MobileNet for fine-tuning
# Assuming 'mobilenet' is part of the model
mobilenet = trmodel.get_layer('mobilenetv2_1.00_224')  # Change this if needed

# Unfreeze the last few layers
for layer in mobilenet.layers[-15:]:  # Adjust number of layers to unfreeze
    layer.trainable = True

# Compile the model again after unfreezing layers
trmodel.compile(
    optimizer=Adam(learning_rate=1e-6),  # You may want to lower the learning rate for fine-tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=18,  # Continue training for 10 more epochs
     initial_epoch=8,
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, lr_schedule]
)



Epoch 9/18
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3849 - loss: 2.2853
Epoch 9: val_loss improved from inf to 1.42494, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 752s 2s/step - accuracy: 0.3848 - loss: 2.2853 - val_accuracy: 0.5945 - val_loss: 1.4249 - learning_rate: 1.0000e-06
Epoch 10/18
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3798 - loss: 2.3143
Epoch 10: val_loss did not improve from 1.42494
295/295 ━━━━━━━━━━━━━━━━━━━━ 758s 2s/step - accuracy: 0.3798 - loss: 2.3143 - val_accuracy: 0.6003 - val_loss: 1.4466 - learning_rate: 1.0000e-06
Epoch 11/18
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3868 - loss: 2.2824
Epoch 11: val_loss improved from 1.42494 to 1.36917, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 798s 3s/step - accuracy: 0.3868 - loss: 2.2824 - val_accuracy: 0.6102 - val_loss: 1.3692 - learning_rate: 1.0000e-06
Epoch 12/18
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - 

In [15]:

from tensorflow.keras.models import load_model

# Load the model from the latest checkpoint
trmodelcontinue = load_model('best_transfer_finetune_model.keras')


In [16]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=6, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

history = trmodelcontinue.fit( 
    normalized_train,
    epochs=20,  # Total 10 more)
    validation_data=normalized_validation,
    callbacks=[early_stopping, model_checkpoint, lr_schedule]
)

Epoch 1/20
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3857 - loss: 2.2903
Epoch 1: val_loss improved from inf to 1.39845, saving model to best_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 560s 2s/step - accuracy: 0.3857 - loss: 2.2903 - val_accuracy: 0.6176 - val_loss: 1.3985 - learning_rate: 1.0000e-06
Epoch 2/20
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3888 - loss: 2.2614
Epoch 2: val_loss did not improve from 1.39845
295/295 ━━━━━━━━━━━━━━━━━━━━ 781s 2s/step - accuracy: 0.3888 - loss: 2.2613 - val_accuracy: 0.6204 - val_loss: 1.4144 - learning_rate: 1.0000e-06
Epoch 3/20
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3810 - loss: 2.2599
Epoch 3: val_loss did not improve from 1.39845
295/295 ━━━━━━━━━━━━━━━━━━━━ 732s 2s/step - accuracy: 0.3810 - loss: 2.2599 - val_accuracy: 0.6020 - val_loss: 1.4131 - learning_rate: 1.0000e-06
Epoch 4/20
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3845 - loss: 2.2826
Epoch 4: val_loss improved fr

In [13]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('best_transfer_finetune_model.keras')

# Unfreeze only the last few layers of MobileNet for fine-tuning
# Assuming 'mobilenet' is part of the model
mobilenet = trmodel.get_layer('mobilenetv2_1.00_224')  # Change this if needed

# Unfreeze the last few layers
for layer in mobilenet.layers[-20:]:  # Adjust number of layers to unfreeze
    layer.trainable = True

# Compile the model again after unfreezing layers
trmodel.compile(
    optimizer=Adam(learning_rate=1e-5),  # You may want to lower the learning rate for fine-tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train, 
    epochs=5, 
    validation_data=normalized_validation, 
    callbacks=[early_stopping, model_checkpoint, lr_schedule]
)


Epoch 1/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3639 - loss: 2.4251
Epoch 1: val_accuracy improved from -inf to 0.61349, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 598s 2s/step - accuracy: 0.3639 - loss: 2.4251 - val_accuracy: 0.6135 - val_loss: 1.3985 - learning_rate: 1.0000e-05
Epoch 2/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3669 - loss: 2.3517
Epoch 2: val_accuracy improved from 0.61349 to 0.63211, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 564s 2s/step - accuracy: 0.3670 - loss: 2.3516 - val_accuracy: 0.6321 - val_loss: 1.3343 - learning_rate: 1.0000e-05
Epoch 3/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3864 - loss: 2.2771
Epoch 3: val_accuracy improved from 0.63211 to 0.64062, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 528s 2s/step - accuracy: 0.3864 - loss: 2.2771 - val_accuracy: 0.6406 - val_loss: 1.3

In [14]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)
trmodelcontinue = load_model('bestaccuracy_transfer_finetune_model.keras')
# Continue training the model
history = trmodelcontinue.fit(
    normalized_train, 
    epochs=5, 
    validation_data=normalized_validation, 
    callbacks=[early_stopping, model_checkpoint,model_checkpoint2, lr_schedule]
)


Epoch 1/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3827 - loss: 2.2748
Epoch 1: val_loss improved from inf to 1.34373, saving model to best_transfer_finetune_model.keras

Epoch 1: val_accuracy improved from -inf to 0.61595, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 553s 2s/step - accuracy: 0.3827 - loss: 2.2749 - val_accuracy: 0.6160 - val_loss: 1.3437 - learning_rate: 1.0000e-05
Epoch 2/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3914 - loss: 2.2701
Epoch 2: val_loss did not improve from 1.34373

Epoch 2: val_accuracy improved from 0.61595 to 0.64047, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 556s 2s/step - accuracy: 0.3914 - loss: 2.2701 - val_accuracy: 0.6405 - val_loss: 1.3690 - learning_rate: 1.0000e-05
Epoch 3/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3920 - loss: 2.2595
Epoch 3: val_loss did not improve from 1.34373

Epoch 3: val_accuracy did not imp

In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)
trmodelcontinue = load_model('bestaccuracy_transfer_finetune_model.keras')
# Continue training the model
history = trmodelcontinue.fit(
    normalized_train, 
    epochs=10, 
    validation_data=normalized_validation, 
    callbacks=[early_stopping, model_checkpoint,model_checkpoint2, lr_schedule]
)


Epoch 1/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3994 - loss: 2.2331
Epoch 1: val_loss improved from inf to 1.35271, saving model to best_transfer_finetune_model.keras

Epoch 1: val_accuracy improved from -inf to 0.64062, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 740s 2s/step - accuracy: 0.3994 - loss: 2.2332 - val_accuracy: 0.6406 - val_loss: 1.3527 - learning_rate: 1.0000e-05
Epoch 2/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3947 - loss: 2.2383
Epoch 2: val_loss did not improve from 1.35271

Epoch 2: val_accuracy did not improve from 0.64062
295/295 ━━━━━━━━━━━━━━━━━━━━ 731s 2s/step - accuracy: 0.3948 - loss: 2.2383 - val_accuracy: 0.6279 - val_loss: 1.3533 - learning_rate: 1.0000e-05
Epoch 3/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4086 - loss: 2.2146
Epoch 3: val_loss improved from 1.35271 to 1.33987, saving model to best_transfer_finetune_model.keras

Epoch 3: val_accuracy did not improve 

In [13]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)
trmodelcontinue = load_model('bestaccuracy_transfer_finetune_model.keras')
# Continue training the model
history = trmodelcontinue.fit(
    normalized_train, 
    epochs=8, 
    validation_data=normalized_validation, 
    callbacks=[early_stopping, model_checkpoint,model_checkpoint2, lr_schedule]
)


Epoch 1/8
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4133 - loss: 2.1411
Epoch 1: val_loss improved from inf to 1.26712, saving model to best_transfer_finetune_model.keras

Epoch 1: val_accuracy improved from -inf to 0.67023, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 459s 1s/step - accuracy: 0.4133 - loss: 2.1411 - val_accuracy: 0.6702 - val_loss: 1.2671 - learning_rate: 5.0000e-06
Epoch 2/8
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4209 - loss: 2.1225
Epoch 2: val_loss did not improve from 1.26712

Epoch 2: val_accuracy did not improve from 0.67023
295/295 ━━━━━━━━━━━━━━━━━━━━ 448s 1s/step - accuracy: 0.4209 - loss: 2.1225 - val_accuracy: 0.6291 - val_loss: 1.3538 - learning_rate: 5.0000e-06
Epoch 3/8
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4228 - loss: 2.0975
Epoch 3: val_loss improved from 1.26712 to 1.22449, saving model to best_transfer_finetune_model.keras

Epoch 3: val_accuracy did not improve fro

In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')

# Unfreeze only the last few layers of MobileNet for fine-tuning
# Assuming 'mobilenet' is part of the model
mobilenet = trmodel.get_layer('mobilenetv2_1.00_224')  # Change this if needed

# Unfreeze the last few layers
for layer in mobilenet.layers[-25:]:  # Adjust number of layers to unfreeze
    layer.trainable = True

# Compile the model again after unfreezing layers
trmodel.compile(
    optimizer=Adam(learning_rate=1e-4),  # You may want to lower the learning rate for fine-tuning
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=10,  # Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)


Epoch 1/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4783 - loss: 1.8622
Epoch 1: val_loss improved from inf to 1.10235, saving model to best_transfer_finetune_model.keras

Epoch 1: val_accuracy improved from -inf to 0.69079, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 443s 1s/step - accuracy: 0.4783 - loss: 1.8622 - val_accuracy: 0.6908 - val_loss: 1.1024 - learning_rate: 1.0000e-04
Epoch 2/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4902 - loss: 1.8022
Epoch 2: val_loss did not improve from 1.10235

Epoch 2: val_accuracy improved from 0.69079 to 0.69231, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 440s 1s/step - accuracy: 0.4901 - loss: 1.8023 - val_accuracy: 0.6923 - val_loss: 1.1194 - learning_rate: 1.0000e-04
Epoch 3/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4921 - loss: 1.7955
Epoch 3: val_loss improved from 1.10235 to 1.05842, saving model to best_trans

In [13]:

from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=20,
    initial_epoch=10,# Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)


Epoch 11/20
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5465 - loss: 1.5769
Epoch 11: val_loss improved from inf to 0.92320, saving model to best_transfer_finetune_model.keras

Epoch 11: val_accuracy improved from -inf to 0.72039, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 438s 1s/step - accuracy: 0.5465 - loss: 1.5770 - val_accuracy: 0.7204 - val_loss: 0.9232 - learning_rate: 1.0000e-04
Epoch 12/20
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5429 - loss: 1.6023
Epoch 12: val_loss improved from 0.92320 to 0.88972, saving model to best_transfer_finetune_model.keras

Epoch 12: val_accuracy improved from 0.72039 to 0.73411, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 665s 2s/step - accuracy: 0.5429 - loss: 1.6022 - val_accuracy: 0.7341 - val_loss: 0.8897 - learning_rate: 1.0000e-04
Epoch 13/20
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5553 - loss: 1.5084
Epoch 13: val_l

In [12]:

from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=30,
    initial_epoch=20,# Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)

Epoch 21/30
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5877 - loss: 1.4019
Epoch 21: val_loss improved from inf to 0.70889, saving model to best_transfer_finetune_model.keras

Epoch 21: val_accuracy improved from -inf to 0.80016, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 763s 2s/step - accuracy: 0.5877 - loss: 1.4018 - val_accuracy: 0.8002 - val_loss: 0.7089 - learning_rate: 1.0000e-04
Epoch 22/30
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5981 - loss: 1.3598
Epoch 22: val_loss did not improve from 0.70889

Epoch 22: val_accuracy did not improve from 0.80016
295/295 ━━━━━━━━━━━━━━━━━━━━ 791s 2s/step - accuracy: 0.5981 - loss: 1.3598 - val_accuracy: 0.7701 - val_loss: 0.8072 - learning_rate: 1.0000e-04
Epoch 23/30
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5976 - loss: 1.3530
Epoch 23: val_loss improved from 0.70889 to 0.70051, saving model to best_transfer_finetune_model.keras

Epoch 23: val_accuracy did not

In [13]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=40,
    initial_epoch=30,# Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)

Epoch 31/40
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6322 - loss: 1.2068
Epoch 31: val_loss improved from inf to 0.61749, saving model to best_transfer_finetune_model.keras

Epoch 31: val_accuracy improved from -inf to 0.82566, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 748s 2s/step - accuracy: 0.6322 - loss: 1.2068 - val_accuracy: 0.8257 - val_loss: 0.6175 - learning_rate: 1.0000e-04
Epoch 32/40
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6505 - loss: 1.1775
Epoch 32: val_loss improved from 0.61749 to 0.59960, saving model to best_transfer_finetune_model.keras

Epoch 32: val_accuracy improved from 0.82566 to 0.82860, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 729s 2s/step - accuracy: 0.6505 - loss: 1.1775 - val_accuracy: 0.8286 - val_loss: 0.5996 - learning_rate: 1.0000e-04
Epoch 33/40
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6485 - loss: 1.1849
Epoch 33: val_l

In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=50,
    initial_epoch=40,# Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)

Epoch 41/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6905 - loss: 1.0271
Epoch 41: val_loss improved from inf to 0.52223, saving model to best_transfer_finetune_model.keras

Epoch 41: val_accuracy improved from -inf to 0.84786, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 699s 2s/step - accuracy: 0.6905 - loss: 1.0271 - val_accuracy: 0.8479 - val_loss: 0.5222 - learning_rate: 1.0000e-04
Epoch 42/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6926 - loss: 1.0337
Epoch 42: val_loss did not improve from 0.52223

Epoch 42: val_accuracy did not improve from 0.84786
295/295 ━━━━━━━━━━━━━━━━━━━━ 698s 2s/step - accuracy: 0.6926 - loss: 1.0338 - val_accuracy: 0.8361 - val_loss: 0.5233 - learning_rate: 1.0000e-04
Epoch 43/50
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6915 - loss: 1.0204
Epoch 43: val_loss did not improve from 0.52223

Epoch 43: val_accuracy did not improve from 0.84786
295/295 ━━━━━━━━━━━━━━━━━━━━ 653s 

In [14]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=60,
    initial_epoch=50,# Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)

Epoch 51/60
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7041 - loss: 0.9833
Epoch 51: val_loss improved from inf to 0.52258, saving model to best_transfer_finetune_model.keras

Epoch 51: val_accuracy improved from -inf to 0.84457, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 537s 2s/step - accuracy: 0.7041 - loss: 0.9833 - val_accuracy: 0.8446 - val_loss: 0.5226 - learning_rate: 1.0000e-04
Epoch 52/60
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7149 - loss: 0.9371
Epoch 52: val_loss did not improve from 0.52258

Epoch 52: val_accuracy did not improve from 0.84457
295/295 ━━━━━━━━━━━━━━━━━━━━ 657s 2s/step - accuracy: 0.7149 - loss: 0.9371 - val_accuracy: 0.8420 - val_loss: 0.5255 - learning_rate: 1.0000e-04
Epoch 53/60
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7158 - loss: 0.9438
Epoch 53: val_loss improved from 0.52258 to 0.51964, saving model to best_transfer_finetune_model.keras

Epoch 53: val_accuracy improve

In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=70,
    initial_epoch=60,# Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)

Epoch 61/70
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7319 - loss: 0.8761
Epoch 61: val_loss improved from inf to 0.41534, saving model to best_transfer_finetune_model.keras

Epoch 61: val_accuracy improved from -inf to 0.88322, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 632s 2s/step - accuracy: 0.7319 - loss: 0.8761 - val_accuracy: 0.8832 - val_loss: 0.4153 - learning_rate: 1.0000e-04
Epoch 62/70
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7523 - loss: 0.8060
Epoch 62: val_loss improved from 0.41534 to 0.37357, saving model to best_transfer_finetune_model.keras

Epoch 62: val_accuracy improved from 0.88322 to 0.89130, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 673s 2s/step - accuracy: 0.7523 - loss: 0.8061 - val_accuracy: 0.8913 - val_loss: 0.3736 - learning_rate: 1.0000e-04
Epoch 63/70
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7424 - loss: 0.8579
Epoch 63: val_l

In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=80,
    initial_epoch=70,# Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)

Epoch 71/80
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7893 - loss: 0.6872
Epoch 71: val_loss improved from inf to 0.34321, saving model to best_transfer_finetune_model.keras

Epoch 71: val_accuracy improved from -inf to 0.88980, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 748s 2s/step - accuracy: 0.7893 - loss: 0.6872 - val_accuracy: 0.8898 - val_loss: 0.3432 - learning_rate: 5.0000e-05
Epoch 72/80
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7814 - loss: 0.7070
Epoch 72: val_loss improved from 0.34321 to 0.30723, saving model to best_transfer_finetune_model.keras

Epoch 72: val_accuracy improved from 0.88980 to 0.90970, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 725s 2s/step - accuracy: 0.7814 - loss: 0.7070 - val_accuracy: 0.9097 - val_loss: 0.3072 - learning_rate: 5.0000e-05
Epoch 73/80
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7930 - loss: 0.6795
Epoch 73: val_l

In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=90,
    initial_epoch=80,# Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)

Epoch 81/90
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8073 - loss: 0.6139
Epoch 81: val_loss improved from inf to 0.26393, saving model to best_transfer_finetune_model.keras

Epoch 81: val_accuracy improved from -inf to 0.93092, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 653s 2s/step - accuracy: 0.8073 - loss: 0.6139 - val_accuracy: 0.9309 - val_loss: 0.2639 - learning_rate: 2.5000e-05
Epoch 82/90
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8116 - loss: 0.5982
Epoch 82: val_loss did not improve from 0.26393

Epoch 82: val_accuracy did not improve from 0.93092
295/295 ━━━━━━━━━━━━━━━━━━━━ 777s 2s/step - accuracy: 0.8116 - loss: 0.5983 - val_accuracy: 0.9130 - val_loss: 0.2995 - learning_rate: 2.5000e-05
Epoch 83/90
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8159 - loss: 0.5873
Epoch 83: val_loss did not improve from 0.26393

Epoch 83: val_accuracy did not improve from 0.93092
295/295 ━━━━━━━━━━━━━━━━━━━━ 745s 

In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Load the previously saved model
trmodel = load_model('bestaccuracy_transfer_finetune_model.keras')
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_transfer_finetune_model.keras', save_best_only=True, monitor='val_loss', verbose=1)
model_checkpoint2 = ModelCheckpoint('bestaccuracy_transfer_finetune_model.keras', save_best_only=True, monitor='val_accuracy', verbose=1)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)

# Continue training the model
history = trmodel.fit(
    normalized_train,  # Your training dataset
    epochs=100,
    initial_epoch=90,# Continue training for 10 more epochs
    validation_data=normalized_validation,  # Your validation dataset
    callbacks=[early_stopping, model_checkpoint, model_checkpoint2, lr_schedule]
)

Epoch 91/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8226 - loss: 0.5694
Epoch 91: val_loss improved from inf to 0.25718, saving model to best_transfer_finetune_model.keras

Epoch 91: val_accuracy improved from -inf to 0.91859, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 776s 2s/step - accuracy: 0.8226 - loss: 0.5694 - val_accuracy: 0.9186 - val_loss: 0.2572 - learning_rate: 2.5000e-05
Epoch 92/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8192 - loss: 0.5680
Epoch 92: val_loss improved from 0.25718 to 0.24729, saving model to best_transfer_finetune_model.keras

Epoch 92: val_accuracy improved from 0.91859 to 0.91890, saving model to bestaccuracy_transfer_finetune_model.keras
295/295 ━━━━━━━━━━━━━━━━━━━━ 867s 3s/step - accuracy: 0.8192 - loss: 0.5680 - val_accuracy: 0.9189 - val_loss: 0.2473 - learning_rate: 2.5000e-05
Epoch 93/100
295/295 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8308 - loss: 0.5582
Epoch 93: va